In [1]:
import re
import os
import glob
import json
import pytesseract
from PIL import Image
import pandas as pd

In [4]:
# My offsets
row_height = 590
rows_start = 330
left_margin_rows = 220
right_margin_rows = 245
box_width = 1500

In [14]:
ls data/1

extract_pipe.py  page_19.png  page_29.png  page_39.png      page_4.png
nohup.log        page_1.png   page_2.png   page_3.png       page_5.png
page_10.png      page_20.png  page_30.png  page_3.png.json  page_6.png
page_11.png      page_21.png  page_31.png  page_40.png      page_7.png
page_12.png      page_22.png  page_32.png  page_41.png      page_8.png
page_13.png      page_23.png  page_33.png  page_42.png      page_9.png
page_14.png      page_24.png  page_34.png  page_43.png      png
page_15.png      page_25.png  page_35.png  page_44.png
page_16.png      page_26.png  page_36.png  page_45.png
page_17.png      page_27.png  page_37.png  page_46.png
page_18.png      page_28.png  page_38.png  page_47.png


In [23]:
regexes = {
    'Name': "(?<=Name)(.*)",
    'Relative': "(Father|Mother|Husband)\\\'s.*?Name.*?:(.*)",
    'Age': "Age:\s([0-9]+)",
    'Sex': "Sex:\s(MALE|FEMALE|THIRD GENDER)"
}
compiled_regexes = {k:re.compile(v) for k,v in regexes.items()}

row_sizes = {
    'row_height': 290,
    'box_width': 752,
}
def get_string_from_row(row):
    '''Takes in a single row (Image Object), gets 3 text strings out'''
    box_width = row_sizes['box_width']
    
    return [pytesseract.image_to_string(row.crop((i * box_width, 0 ,(i + 1) * box_width, row.size[1]))) 
            for i in range(3)]

def get_first_offsets(width, height):
    # First page offsets
    return {
    #     'name and reservation status of parliamentary constituencies': (0, 300, width, 450),
        'Sections': (0, 1060, width/2-200, 1800), # Sections in PDF
        'Details': (width/2+200,1060, width, 1800), # town,ward,police,tehsil,district,pin
        'Polling Station':(0,2000, width/2+50, height-1050), # no and name of polling station
        'Voters': (width/6+500,height-950,width, height-800)
    }
        
def get_voter_offsets(width, height):
    return {
        'header': (0,0,width, 200),
        'rows': (101, 210, width-120, height-370), # As close to row lines as possible
    }

def handle_sheet(filename):
    '''Takes a filename and returns a list of strings inside it'''
    image = Image.open(filename)
    width, height = image.size
    if os.path.basename(filename) == 'page_1.png':
        return {
            k : pytesseract.image_to_string(image.crop(v)) for k,v in get_first_offsets(width, height).items()
        }
    else:
#         import pdb; pdb.set_trace()
        header, all_rows = (image.crop(v) for k, v in get_voter_offsets(width, height).items())
        rows = [all_rows.crop((0, i * row_sizes['row_height'], all_rows.size[0], (i+1) * row_sizes['row_height'])) for i in range(10)] 
        strings = [get_string_from_row(row) for row in rows]
        all_strings = [item for sublist in strings for item in sublist]
        regexes = [[regex.search(string_)[0] if regex.search(string_) else None for regex in compiled_regexes.values()] for string_ in all_strings]
        return{
            'Header': pytesseract.image_to_string(header),
#             'Footer': pytesseract.image_to_string(image.crop(footer_box)),
#             'Rows': rows,
            'Strings': strings,
            "Regexes": regexes
        }

In [24]:
handle_sheet('data/42/page_3.png')

{'Header': 'Assembly Constituency No and Name: 15-SHAKUR BASTI Part No.- : 42\n\nSection No and Name : 1-BLOCK BG-6,PASCHIM VIHAR',
 'Strings': [["ACB1721132\n\n \n\nName: VINAY SHARMA\n\nFather's Name : KANHAIYALAL\nSHARMA Photo is\nHouse Number: 163C Available\nAge: 40 Sex: MALE",
   "ACB1756096\n\n \n\nName: PUNEET KAUR\n\nHusband's Name : GURDEEP SINGH\nHouse Number: 208A\n\nAge: 30 Sex: FEMALE\n\nPhoto is\n\nAvailable",
   "ACB1813013\n\n \n\nName: yashika khandelwal\nFather's Name: sanjay khandelwal\nHouse Number: 216-B\n\nAge: 19 Sex: FEMALE\n\nPhoto is\n\nAvailable"],
  ["Name : CHANDER SHEKHAR\nFather's Name : NIHAL SINGH\nHouse Number : 220-C\nAge: 37 Sex: MALE\n\nACB1705565\n\nPhoto is\n\nAvailable",
   "Name : HITESH ANAND\nFather's Name: RAJIV ANAND\nHouse Number : 223-B\n\nAge: 22 Sex: MALE\n\nACB1776533\n\nPhoto is\n\nAvailable",
   "ACB1740983\nName: RAMA BATRA\n\nHusband's Name : SHYAM SUNDER\nBATRA Photo is\nHouse Number : 226C\nAge: 56 Sex: FEMALE\n\nAvailable"],
  [

In [251]:
'''This is a dumb, lazy approach'''
num_files = sorted([int(re.search('[0-9]+', filename)[0]) for filename in glob.glob('data/*/*.png')]);
num_files
# for file in usable_files:
#     with open(file+'.json', 'w') as f:
#         json.dump(handle_sheet(file), f)

TypeError: int() argument must be a string, a bytes-like object or a number, not 're.Match'

In [106]:
num_files = sorted([int(re.search('[0-9]+', filename)[0]) for filename in glob.glob('./*.json')])
dataframes = []
for file in num_files:
    with open(f'./page_{file}.png.json') as f:
        x = json.load(f)
        dataframes += [pd.DataFrame(x['Regexes'])]

In [15]:
df = pd.concat(dataframes).dropna(how='all')
df.columns = ['Name', 'Relative', 'Age', 'Sex']
df['Name'] = df['Name'].apply(lambda x: x.split(':')[-1].strip() if isinstance(x, str) else None)
df['Age'] = df['Age'].apply(lambda x: x.split(':')[-1].strip() if isinstance(x, str) else None)
df['Sex'] = df['Sex'].apply(lambda x: x.split(':')[-1].strip() if isinstance(x, str) else None)


In [17]:
df.to_csv('./first_pdf.csv', index=None)

In [6]:
import glob
import json
import pandas as pd


In [5]:
def get_json(filename):
    with open(filename) as file:
        return json.load(file)

In [24]:
sheets = [(get_json(filename), filename) for filename in glob.glob('./data/*/*.json') if os.path.basename(filename)!= 'page_1.png.json']

In [25]:
sheets[0]

({'Header': 'Assembly Constituency No and Name: 15-SHAKUR BASTI Part No.- : 111\n\nSection No and Name : 1-HARI SINGH PARK,NEW MULTAN NAGAR\n\n \n\n \n\n \n\nBAW APmnocagca BAD mian van vococn BAD APmnAorATOC',
  'Strings': [["Name: DEEPAK\n\nFather's Name: KEWAL KRISHAN\nHouse Number: 87\n\nPhoto is\nAge: 45 Sex: MALE\n\nAvailable\n\n \n\n \n\n \n\nToa]\n\nACRBNSK6ATO2R",
    "Name: REENA\n\nHusband's Name : DEEPAK\nHouse Number: 87 Photo is\nAge: 40 Sex: FEMALE\n\nAvailable\n\n \n\n \n\n \n\n545 ACRNS6AaNN",
    "Name: SURJEET SINGH\n\nFather's Name : GURCHARAN SINGH\nHouse Number: 88\n\nPhoto is\nAge: 76 Sex: MALE\n\nAvailable\n\n \n\n \n\n \n\n546 ACRNS6A210"],
   ["| el\n\nName : SURENDER KAUR\nHusband's Name: SURJEET SINGH\nHouse Number: 88\n\nAge: 74 Sex: FEMALE\n\nPhoto is\nAvailable\n\nAMrDRDNHORAQNT",
    "| El\n\nName : SURESH CHANDER\nFather's Name : TOTARAM\nHouse Number: 88\n\nAge: 68 Sex: MALE\n\nPhoto is\nAvailable\n\nAPrR4A2ADOONCT",
    "| el\n\nName: RASHMI\nHusband'

In [23]:
len(sheets)

5109

In [26]:
dataframes = []
for sheet, name in sheets:
    new_df = pd.DataFrame(sheet['Regexes'])
    new_df['Section'] = sheet['Header']
#     print(new_df.size, len(sheet['Strings']))
    new_df['Strings'] = [item for sublist in sheet['Strings'] for item in sublist]
    new_df['Sheet'] = name
    dataframes += [new_df]
    

In [27]:
one_four_three = pd.concat(dataframes)

In [28]:
one_four_three.head()

,0,1,2,3,Section,Strings,Sheet
0,: DEEPAK,Father's Name: KEWAL KRISHAN,Age: 45,Sex: MALE,Assembly Constituency No and Name: 15-SHAKUR B...,Name: DEEPAK\n\nFather's Name: KEWAL KRISHAN\n...,./data/111/page_21.png.json
1,: REENA,Husband's Name : DEEPAK,Age: 40,Sex: FEMALE,Assembly Constituency No and Name: 15-SHAKUR B...,Name: REENA\n\nHusband's Name : DEEPAK\nHouse ...,./data/111/page_21.png.json
2,: SURJEET SINGH,Father's Name : GURCHARAN SINGH,Age: 76,Sex: MALE,Assembly Constituency No and Name: 15-SHAKUR B...,Name: SURJEET SINGH\n\nFather's Name : GURCHAR...,./data/111/page_21.png.json
3,: SURENDER KAUR,Husband's Name: SURJEET SINGH,Age: 74,Sex: FEMALE,Assembly Constituency No and Name: 15-SHAKUR B...,| el\n\nName : SURENDER KAUR\nHusband's Name: ...,./data/111/page_21.png.json
4,: SURESH CHANDER,Father's Name : TOTARAM,Age: 68,Sex: MALE,Assembly Constituency No and Name: 15-SHAKUR B...,| El\n\nName : SURESH CHANDER\nFather's Name :...,./data/111/page_21.png.json


In [29]:
one_four_three.shape

(148980, 7)

In [30]:
one_four_three.columns = ['Name', 'Relative', 'Age', 'Sex', 'Section', 'Raw String', 'Sheet']

In [31]:
df = one_four_three

In [32]:
df['Name'] = df['Name'].apply(lambda x: x.split(':')[-1].strip() if isinstance(x, str) else None)

In [33]:
df.head()

,Name,Relative,Age,Sex,Section,Raw String,Sheet
0,DEEPAK,Father's Name: KEWAL KRISHAN,Age: 45,Sex: MALE,Assembly Constituency No and Name: 15-SHAKUR B...,Name: DEEPAK\n\nFather's Name: KEWAL KRISHAN\n...,./data/111/page_21.png.json
1,REENA,Husband's Name : DEEPAK,Age: 40,Sex: FEMALE,Assembly Constituency No and Name: 15-SHAKUR B...,Name: REENA\n\nHusband's Name : DEEPAK\nHouse ...,./data/111/page_21.png.json
2,SURJEET SINGH,Father's Name : GURCHARAN SINGH,Age: 76,Sex: MALE,Assembly Constituency No and Name: 15-SHAKUR B...,Name: SURJEET SINGH\n\nFather's Name : GURCHAR...,./data/111/page_21.png.json
3,SURENDER KAUR,Husband's Name: SURJEET SINGH,Age: 74,Sex: FEMALE,Assembly Constituency No and Name: 15-SHAKUR B...,| el\n\nName : SURENDER KAUR\nHusband's Name: ...,./data/111/page_21.png.json
4,SURESH CHANDER,Father's Name : TOTARAM,Age: 68,Sex: MALE,Assembly Constituency No and Name: 15-SHAKUR B...,| El\n\nName : SURESH CHANDER\nFather's Name :...,./data/111/page_21.png.json


In [34]:
df['Age'] = df['Age'].apply(lambda x: x.split(':')[-1].strip() if isinstance(x, str) else None)
df['Sex'] = df['Sex'].apply(lambda x: x.split(':')[-1].strip() if isinstance(x, str) else None)


In [35]:
df.head()

,Name,Relative,Age,Sex,Section,Raw String,Sheet
0,DEEPAK,Father's Name: KEWAL KRISHAN,45,MALE,Assembly Constituency No and Name: 15-SHAKUR B...,Name: DEEPAK\n\nFather's Name: KEWAL KRISHAN\n...,./data/111/page_21.png.json
1,REENA,Husband's Name : DEEPAK,40,FEMALE,Assembly Constituency No and Name: 15-SHAKUR B...,Name: REENA\n\nHusband's Name : DEEPAK\nHouse ...,./data/111/page_21.png.json
2,SURJEET SINGH,Father's Name : GURCHARAN SINGH,76,MALE,Assembly Constituency No and Name: 15-SHAKUR B...,Name: SURJEET SINGH\n\nFather's Name : GURCHAR...,./data/111/page_21.png.json
3,SURENDER KAUR,Husband's Name: SURJEET SINGH,74,FEMALE,Assembly Constituency No and Name: 15-SHAKUR B...,| el\n\nName : SURENDER KAUR\nHusband's Name: ...,./data/111/page_21.png.json
4,SURESH CHANDER,Father's Name : TOTARAM,68,MALE,Assembly Constituency No and Name: 15-SHAKUR B...,| El\n\nName : SURESH CHANDER\nFather's Name :...,./data/111/page_21.png.json


In [36]:
df['Relative_']=df['Relative'].apply(lambda x: x.split(':')[0] if isinstance(x, str)else None)
df['Relative Name']=df['Relative'].apply(lambda x: x.split(':')[1] if isinstance(x, str)else None)

In [37]:
df.drop('Relative', axis=1, inplace=True)

In [38]:
df = df.rename(columns={'Relative_': 'Relative'})

In [39]:
df['Section'] = df['Section'].apply(lambda x: x.split(':')[-1] if isinstance(x, str) else None)

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 148980 entries, 0 to 29
Data columns (total 8 columns):
Name             143298 non-null object
Age              142789 non-null object
Sex              143137 non-null object
Section          148980 non-null object
Raw String       148980 non-null object
Sheet            148980 non-null object
Relative         143095 non-null object
Relative Name    143095 non-null object
dtypes: object(8)
memory usage: 10.2+ MB


In [45]:
import re

df['Voter ID'] = df['Raw String'].apply(lambda x: re.search('([A-Z,0-9])+', x.split('Name')[0]))

In [48]:
df['Voter Id'] = df['Voter ID'].apply(lambda x: x[0] if x else None)

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 148980 entries, 0 to 29
Data columns (total 10 columns):
Name             143298 non-null object
Age              142789 non-null object
Sex              143137 non-null object
Section          148980 non-null object
Raw String       148980 non-null object
Sheet            148980 non-null object
Relative         143095 non-null object
Relative Name    143095 non-null object
Voter ID         101458 non-null object
Voter Id         101458 non-null object
dtypes: object(10)
memory usage: 12.5+ MB


In [51]:
df.drop('Voter ID', inplace=True, axis=1)

In [56]:
temp = df.dropna(thresh=6)

In [57]:
temp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 143298 entries, 0 to 29
Data columns (total 9 columns):
Name             143298 non-null object
Age              142789 non-null object
Sex              143137 non-null object
Section          143298 non-null object
Raw String       143298 non-null object
Sheet            143298 non-null object
Relative         143095 non-null object
Relative Name    143095 non-null object
Voter Id         101458 non-null object
dtypes: object(9)
memory usage: 10.9+ MB


In [22]:
temp.fillna('na').to_excel('./total.xlsx')

In [59]:
from sqlalchemy import create_engine

In [60]:
engine = create_engine('sqlite:///shakurbasti.db')

In [67]:
temp.fillna('').to_sql('voters', engine, index=None, if_exists='replace')

In [68]:
engine.table_names()

['voters', 'voters2']

In [69]:
df = pd.read_sql('select * from voters', engine)

In [74]:
df['Relative'] = df['Relative'].apply(lambda x: re.sub("\'s", '', x.split('Name')[0]).upper())

In [75]:
df.Sex.values

,Name,Age,Sex,Section,Raw String,Sheet,Relative,Relative Name,Voter Id
0,DEEPAK,45,MALE,"1-HARI SINGH PARK,NEW MULTAN NAGAR\n\n \n\n \...",Name: DEEPAK\n\nFather's Name: KEWAL KRISHAN\n...,./data/111/page_21.png.json,FATHER,KEWAL KRISHAN,
1,REENA,40,FEMALE,"1-HARI SINGH PARK,NEW MULTAN NAGAR\n\n \n\n \...",Name: REENA\n\nHusband's Name : DEEPAK\nHouse ...,./data/111/page_21.png.json,HUSBAND,DEEPAK,
2,SURJEET SINGH,76,MALE,"1-HARI SINGH PARK,NEW MULTAN NAGAR\n\n \n\n \...",Name: SURJEET SINGH\n\nFather's Name : GURCHAR...,./data/111/page_21.png.json,FATHER,GURCHARAN SINGH,
3,SURENDER KAUR,74,FEMALE,"1-HARI SINGH PARK,NEW MULTAN NAGAR\n\n \n\n \...",| el\n\nName : SURENDER KAUR\nHusband's Name: ...,./data/111/page_21.png.json,HUSBAND,SURJEET SINGH,
4,SURESH CHANDER,68,MALE,"1-HARI SINGH PARK,NEW MULTAN NAGAR\n\n \n\n \...",| El\n\nName : SURESH CHANDER\nFather's Name :...,./data/111/page_21.png.json,FATHER,TOTARAM,E


In [48]:
df.fillna('na').to_sql('voters2', engine, index=None)

In [49]:
engine.table_names()

['voters', 'voters2']

In [50]:
ls

app/   extract_pipe.ipynb  shakurbasti.db               total.xlsx
data/  extract_pipe.py     total_slightly_cleaner.xlsx  Untitled.ipynb


In [51]:
!cp shakurbasti.db ../vsa/assets/data/